# Notebook 02 - Data Cleaning

## Objectives

* Check data for duplicate entries
* Determine whether there is any missing data
* Assess column names for inconsistencies and determine how best to rename these
* Assess whether any data needs to have its type changed or be aggregated or renamed
* Consider whether any summary columns should be created to help determine correlation and PPS
* Clean data
* Develop a data cleaning pipeline for use on future data provided by the client

## Inputs

* CSV file generated in previous notebook: outputs/datasets/collection/credit_card_data.csv

## Outputs

* Cleaned data, to be stored in new folder outputs/datasets/cleaned/v1
* Data cleaning pipeline

## Conclusions

* The data had no duplicate entries
* The column `ID` was dropped since it is a unique customer identifier and has no relevance to the target variable
* Column names were edited for consistency and clarity, as well as to conform to naming conventions
* The data relating to customers' sex, education level and marital status was clarified and in doing so its `Dtype` changed so that it was recorded as an object rather than an integer
* Since one of our hypotheses is that past default behaviour will be a strong indicator of future default behaviour, summary columns relating to previous default behaviour were created so that these can be tested for correlation and PPS in the data analysis notebook.


---

# Change working directory

* This notebook is stored in the `jupyter_notebooks` subfolder
* The current working directory therefore needs to be changed to the workspace, i.e., the working directory needs to be changed from the current folder to its parent folder

Firstly, the current directory is accessed with os.getcwd()

In [33]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\franc\\credit-card-default'

Next, the working directory is set as the parent of the current `jupyter_notebooks` directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory
* This allows access to all the files and folders within the workspace, rather than solely those within the `jupyter_notebooks` directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Finally, confirm that the new current directory has been successfully set

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\franc\\credit-card-default'

---

# Load data

The data is loaded from the outputs/datasets/collection folder:

In [45]:
import pandas as pd
df = pd.read_csv('outputs/datasets/collection/credit_card_data.csv')
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


---

# Data Examination

## Check for duplicate entries

Firstly, we want to check the data for any duplicate entries.
* We know that the field `ID` corresponds to the customer ID
* Unless a customer holds multiple credit cards, it should not therefore be possible for a customer to appear in the dataset multiple times

In [46]:
df[df.duplicated(subset='ID')]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month


We see that there are no duplicate values recorded for `ID` and therefore we can assume that there are no duplicate values recorded in the dataset.
* Since `ID` is a unique customer identifier and does not convey any meaning that could impact the target variable, it can be dropped from the dataset.

## Check for missing data

Next, we check whether there is any missing data:

In [47]:
vars_with_missing_data = df.columns[df.isna().any()].tolist()
vars_with_missing_data

[]

There is no missing data in the dataset and so data imputation will not be necessary.

## Check column names

For consistency, it would be best for column names to use snake_case rather than any other naming convention.
* The column names at present are in capitals, and the target variable uses dots rather than underscores:

In [48]:
df.columns.to_list()

['ID',
 'LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'default.payment.next.month']

Viewing all the column names, it seems that it would also be helpful to rename them to improve consistency.
* `PAY_0` relates to September 2005 repayment status, whereas this month's statement amount and previous payment amount are `BILL_AMT1` and `PAY_AMT1` (i.e., the latter two use a 1 rather than a 0)
* It may also be useful to provide clearer naming to ensure clarity and avoid confusion.  
  
This will be addressed by defining a helper function to rename the columns and then calling it from within the data cleaning pipeline.

## Check data types are appropriate and assess whether any data should be aggregated

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

The columns `SEX`, `EDUCATION` and `MARRIAGE` are currently encoded as `int64`.
* These would be more appropriately represented by objects.
* It might be clearer to represent these data by their category names: sex as male or female, rather than 1 or 2, for example.
* This will also allow for combination into a smaller number of categories; the data information table tells us that both 5 and 6 in the `EDUCATION` column correspond to 'unknown'.

Again, this can be done by creating a helper function that is called from within the data cleaning pipeline.

## Consider the creation of summary columns relating to past default behaviour

We have hypothesised that past default behaviour will be a strong indicator of future default behaviour
* The columns `PAY_0` to `PAY_6` relate to default behaviour by month for a six-month period
* However, it may be easier to show correlation if we consider this past six-month period as a whole, rather than examining the data for a correlation between default behaviour in any given month and default in the future
* As such, it makes sense to create further columns and the following columns are created in the pipeline to be examined for correlation:
    - A binary yes / no as to whether the customer has had any default behaviour in the period
    - A crude measure of quantifying default behaviour by summing all positive values in the columns (negative values relate to non-default behaviour and should not be included)
    - A crude measure of the most severe example of any default behaviour by taking the maximum value from these columns. For example, a payment that is 4 months late would yield a 4 in this column, whereas 4 separate delays of 1 month would produce a 1. This may help us to determine whether a customer who is (for example) 4 months late in paying is more or less likely to default in the future than a customer who misses a number of separate payments.

---

# Data Cleaning Pipeline

To summarise the sections above, the steps to be undertaken within the data cleaning pipeline are:
* Drop `ID` from the dataset
* Rename the columns, both for clarity and consistency and to conform to snake_case naming convention
* Replace numerical data representing objects with their object names, combining numbers where appropriate
* Create three further columns named `any_default`, `total_default` and `greatest_default` that summarise customers' previous default behaviour
  
This is done here by creating a single data cleaning pipeline, so that future data can be easily managed in the same way.
* First, the functions `rename_columns`, `clean_categorical_data` and `default_summary` are defined, so that they can be called from within the pipeline
* Within the pipeline:
    - we first use `DropFeatures` to drop `ID` from the dataset
    - we then rename the remaining columns using `FunctionTransformer` and calling the `rename_columns` function
    - the categorical data is replaced by using `FunctionTransformer` and calling the `clean_categorical_data` function
    - finally, the three new columns are created by using `FunctionTransformer` and calling the `default_summary` function



In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from feature_engine.selection import DropFeatures

def rename_columns(X):
    X.columns = ['credit_limit', 'sex', 'education', 'marital_status',
                 'age', 'late_sep', 'late_aug', 'late_jul', 'late_jun',
                 'late_may', 'late_apr', 'bill_sep', 'bill_aug',
                 'bill_jul', 'bill_jun', 'bill_may', 'bill_apr',
                 'prev_payment_sep', 'prev_payment_aug',
                 'prev_payment_jul', 'prev_payment_jun',
                 'prev_payment_may', 'prev_payment_apr',
                 'default_next_month']
    return X


def clean_categorical_data(X):
    sex_mapping = {1: "male", 2: "female"}
    education_mapping = {1: "graduate school", 2: "university", 3: "high school", 4: "other", 5: "unknown", 6: "unknown"}
    marital_status_mapping = {0: "unknown", 1: "married", 2: "single", 3: "other"}
    X['sex'] = X['sex'].replace(sex_mapping)
    X['education'] = X['education'].replace(education_mapping)
    X['marital_status'] = X['marital_status'].replace(marital_status_mapping)
    return X

def default_summary(X):
    X['any_default'] = X[['late_sep', 'late_aug', 'late_jul', 'late_jun',
                 'late_may', 'late_apr']].gt(0).any(axis=1).astype(int)
    X['total_default'] = X[['late_sep', 'late_aug', 'late_jul', 'late_jun',
                 'late_may', 'late_apr']].clip(lower=0).sum(axis=1)
    X['greatest_default'] = X[['late_sep', 'late_aug', 'late_jul', 'late_jun',
                 'late_may', 'late_apr']].clip(lower=0).max(axis=1)
    return X

data_cleaning_pipeline = Pipeline([
      ('drop',  DropFeatures(features_to_drop=['ID'])),
      ('rename_columns', FunctionTransformer(rename_columns, validate=False)),
      ('clean_categorical_data', FunctionTransformer(clean_categorical_data)),
      ('add_default_summary', FunctionTransformer(default_summary)),
])


After creating the pipeline, we can now apply it to the dataframe:

In [51]:
df = data_cleaning_pipeline.fit_transform(df)

Finally, we check the dataframe to ensure that the pipeline has been applied successfully:

In [52]:
df.head()

,credit_limit,sex,education,marital_status,age,late_sep,late_aug,late_jul,late_jun,late_may,...,prev_payment_sep,prev_payment_aug,prev_payment_jul,prev_payment_jun,prev_payment_may,prev_payment_apr,default_next_month,any_default,total_default,greatest_default
0,20000.0,female,university,married,24,2,2,-1,-1,-2,...,0.0,689.0,0.0,0.0,0.0,0.0,1,1,4,2
1,120000.0,female,university,single,26,-1,2,0,0,0,...,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,1,4,2
2,90000.0,female,university,single,34,0,0,0,0,0,...,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,0,0,0
3,50000.0,female,university,married,37,0,0,0,0,0,...,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,0,0,0
4,50000.0,male,university,married,57,-1,0,-1,0,0,...,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,0,0,0


For completeness, we can also use `info` to view the column types and check that the columns `sex`, `education` and `marital_status` now have a `Dtype` of object:

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   credit_limit        30000 non-null  float64
 1   sex                 30000 non-null  object 
 2   education           30000 non-null  object 
 3   marital_status      30000 non-null  object 
 4   age                 30000 non-null  int64  
 5   late_sep            30000 non-null  int64  
 6   late_aug            30000 non-null  int64  
 7   late_jul            30000 non-null  int64  
 8   late_jun            30000 non-null  int64  
 9   late_may            30000 non-null  int64  
 10  late_apr            30000 non-null  int64  
 11  bill_sep            30000 non-null  float64
 12  bill_aug            30000 non-null  float64
 13  bill_jul            30000 non-null  float64
 14  bill_jun            30000 non-null  float64
 15  bill_may            30000 non-null  float64
 16  bill

---

# Push files to Repo

The cleaned data is now saved in a new `outputs/datasets/cleaned/v1` folder.

In [54]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned/v1')
except Exception as e:
  print(e)

df.to_csv(f"outputs/datasets/cleaned/v1/credit_card_data_cleaned.csv", index=False)


[WinError 183] Cannot create a file when that file already exists: 'outputs/datasets/cleaned/v1'


Additionally, the data cleaning pipeline is saved so that it can be reused to clean new data as it is received from the client.
* The pipeline is saved in a new folder, `outputs/ml_pipeline/data_cleaning/v1`

In [55]:
import joblib

filepath = f"outputs/ml_pipeline/data_cleaning/v1"

try:
    os.makedirs(name=filepath)
except Exception as e:
    print(e)

joblib.dump(value=data_cleaning_pipeline, filename=f"{filepath}/data_cleaning_pipeline.pkl")

[WinError 183] Cannot create a file when that file already exists: 'outputs/ml_pipeline/data_cleaning/v1'


['outputs/ml_pipeline/data_cleaning/v1/data_cleaning_pipeline.pkl']